In [178]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
import json
import pandas as pd
import warnings
import time 
import platform
import pymysql
import pandas as pd

def bus_id_from_db():
    conn = pymysql.connect(host="localhost",user="root",password="qkqh1234!",database="finalproject")

    cur = conn.cursor()

    sql = "SELECT bus_id,restaurant_name FROM restaurant"

    cur.execute(sql)

    data = cur.fetchall()

    conn.close()

    return pd.DataFrame(data)


def parsing_review(places):
    url = "https://pcmap-api.place.naver.com/graphql"

    headers = {'Content-Type':"application/json"
        ,"accept":"*/*",'accept-encoding':'gzip, deflate, br','accept-language':"ko","User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36"}

    res = pd.concat([pd.DataFrame(json.loads(requests.post(headers=headers, url=url, 
        data=json.dumps({
            "operationName" : "getVisitorReviewPhotosInVisitorReviewTab",
            "query" : "query getVisitorReviewPhotosInVisitorReviewTab($businessId: String!, $businessType: String, $page: Int, $size: Int, $theme: String, $item: String) {\n  visitorReviews(input: {businessId: $businessId, businessType: $businessType, page: $page, size: $size, theme: $theme, item: $item, isPhotoUsed: true, includeReceiptPhotos: false}) {\n    items {\n      id\n      rating\n      author {\n        id\n        nickname\n        from\n        imageUrl\n        objectId\n        url\n        __typename\n      }\n      body\n      thumbnail\n      media {\n        type\n        thumbnail\n        __typename\n      }\n      tags\n      status\n      visited\n      originType\n      item {\n        name\n        code\n        options\n        __typename\n      }\n      businessName\n      isFollowing\n      visitCount\n      votedKeywords {\n        code\n        iconUrl\n        iconCode\n        displayName\n        __typename\n      }\n      __typename\n    }\n    starDistribution {\n      score\n      count\n      __typename\n    }\n    hideProductSelectBox\n    total\n    __typename\n  }\n}\n",

            "variables" : {
                "businessId" : "{}".format(bus_id),
                "businessType": "restaurant",
                "item": "0",
                "page": 1,
                "size": 20}
        }
    )).text)['data']['visitorReviews']['items'])[['body','thumbnail','originType','visitCount','businessName']] for bus_id in places.iloc[:5,0]]).reset_index(drop=True)

    res.to_csv('./review_data.csv')

    return res

def reviewing_bot(places,review_data):

    headers = {'Content-Type':"application/json"
        ,"accept":"*/*",'accept-encoding':'gzip, deflate, br','accept-language':"ko","User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36"}

    if "mac" in platform.platform():
        driver = webdriver.Chrome("./chromedriver")
    else:
        driver = webdriver.Chrome("./chromedriver.exe")


    driver.get("http://localhost:3000/login")

    for i in range(1,2):
        driver.find_element(By.CSS_SELECTOR,
        "#root > div.SignUp_container__tfO9- > div > div > div.SignUp_loginInfo__IKYx6 > a:nth-child(1)").click()

        driver.find_element(By.CSS_SELECTOR,
        "#root > div > div > div.SignUp_content__WR\+nJ > form > input:nth-child(2)").send_keys(f"test_babo_{i}")

        driver.find_element(By.CSS_SELECTOR,
        "#root > div > div > div.SignUp_content__WR\+nJ > form > input:nth-child(6)").send_keys(f"test_{i}@naver.com")


        driver.find_element(By.CSS_SELECTOR,
        "#root > div > div > div.SignUp_content__WR\+nJ > form > input:nth-child(11)").send_keys("12345678")
        driver.find_element(By.CSS_SELECTOR,
        "#root > div > div > div.SignUp_content__WR\+nJ > form > input:nth-child(16)").send_keys("12345678")

        driver.find_element(By.CSS_SELECTOR,
        "#root > div > div > div.SignUp_content__WR\+nJ > form > div > button").click()

        time.sleep(0.2)
        driver.find_element(By.CSS_SELECTOR,
        "#root > div > div > div.Welcome_buttons__4jqHU > button:nth-child(1)").click()

        # 로그인 수행
        driver.find_element(By.CSS_SELECTOR,
        "#root > div.SignUp_container__tfO9- > div > div > form > input:nth-child(2)").send_keys(f"test_{i}@naver.com")
        driver.find_element(By.CSS_SELECTOR,
        "#root > div.SignUp_container__tfO9- > div > div > form > input:nth-child(6)").send_keys("12345678")

        driver.find_element(By.CSS_SELECTOR,
        "#root > div.SignUp_container__tfO9- > div > div > form > div > button").click()
        
        time.sleep(0.3)
        # 마이페이지 -> 메인 페이지
        driver.find_element(By.CSS_SELECTOR, 
        "#root > div > div:nth-child(1) > div.Logobar_container__E-uvF > div > a").click()

        for place in places.iloc[:,1]:
            driver.find_element(By.CSS_SELECTOR, 
            "#root > div > div > div.Header_background__aIS43 > div.Header_search__78Y2t > form > input").send_keys(place)
            driver.find_element(By.CSS_SELECTOR, 
            "#root > div > div > div.Header_background__aIS43 > div.Header_search__78Y2t > form > button").click()

            
            driver.find_element(By.CSS_SELECTOR,
            "#root > div > div.ListCard_wrapper__Gkssx > a > div > img").click()

            driver.find_element(By.CSS_SELECTOR,
            "#res-reviews > a").click()

            # review_title
            driver.find_element(By.CSS_SELECTOR,
            "#title").send_keys("hi")
            
            # review_content
            driver.find_element(By.CSS_SELECTOR,
            "#image").send_keys("hihi44545454545656565656aaa")

            # image_url
            driver.find_element(By.CSS_SELECTOR,
            "#comment").send_keys("hihi44545454545656565656")

            driver.find_element(By.CSS_SELECTOR,
            "#root > div > div.ReviewWrite_container__7s5NG > form > div:nth-child(9) > button").click()

            # review 1개 작성완료. -> 리뷰작성완료 alert창

            alert = driver.switch_to.alert
            alert.accept()

            # detail => main_page
            driver.find_element(By.CSS_SELECTOR,
            "#root > div > div.Navbar_container__yPJt0 > div:nth-child(1) > a").click()
        



def review_program():
    places = bus_id_from_db()

    parsing_review(places)

    review_data = pd.read_csv('./review_data.csv').drop(columns=['Unnamed: 0'])

    reviewing_bot(places,review_data)


In [179]:
review_program()

/var/folders/d1/5yg0q8_x0_s4wn9j6q1k_4n40000gn/T/ipykernel_1147/627108675.py:58: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("./chromedriver")


In [184]:
df = pd.read_csv('./review_data.csv').drop(columns=['Unnamed: 0'])

df.loc[:,'businessName']

0     낙원갈비집 합정점
1     낙원갈비집 합정점
2     낙원갈비집 합정점
3     낙원갈비집 합정점
4     낙원갈비집 합정점
        ...    
95         부흥가든
96         부흥가든
97         부흥가든
98         부흥가든
99         부흥가든
Name: businessName, Length: 100, dtype: object